In [1]:
import torch
import torch.nn as nn
from torch.optim import Adam
from models.new_wave import WaveNet
from disk_utils import save_model, load_model
from audio_dataset import build_audio_data_loaders

In [2]:
test_size = 0.0
train_dl, _ = build_audio_data_loaders(test_size=test_size)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cpu = torch.device("cpu")

model = WaveNet(residual_channels=64,
                skip_channels=128,
                num_blocks=2,
                num_layers=10,
                kernel_size=2)
criterion = nn.MSELoss()
optimizer = Adam(model.parameters(), lr=2e-4)

In [4]:
num_epochs = 1

In [5]:
model = model.to(device)
model.train()
for epoch in range(num_epochs):
    num_batches = 0
    running_loss = 0.0
    for X, Y, _, _ in train_dl:
        X = X.to(device)
        Y = Y.to(device)

        # Forward pass
        outputs = model(X)
        loss = criterion(outputs, Y)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        num_batches += 1
        if num_batches % 100 == 0:
            print(f"\t{num_batches}\t B.L: {loss.item():.6f}")

    print(f"E {epoch + 1}\t E.L: {running_loss / num_batches:.6f}")

	100	 B.L: 0.229590
	200	 B.L: 0.179496
	300	 B.L: 0.162295
	400	 B.L: 0.189888
E 1	 E.L: 0.180755


In [6]:
model.to(cpu)
save_model(model, "wavenet")